<a href="https://colab.research.google.com/github/tamojeetroychowdhury/Facial-Expression-Recognition/blob/main/Week-2/FER_Neural_Net_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

df = pd.read_csv('gdrive/My Drive/fer2013.csv')

In [3]:
df = df.drop(df[df.emotion == 1].index)

In [37]:
all_labels = np.array(df['emotion'][:35000])
all_labels[np.where(all_labels == 6)] = 1   
all_labels = torch.tensor(all_labels)
#labels_wo_1 = all_labels[np.where(all_labels != 1)]

In [36]:
len(all_labels)

35000

In [5]:
ds = []
for s in df['pixels'][:35000]:
    m = s.split()
    l = np.zeros((48,48))
    for i in range(len(m)):
        n = int(m[i])
        l[i//48][i%48] = n
    ds.append([l])

In [6]:
ds = np.array(ds, dtype = np.float32)
dataset = torch.tensor(ds)
#dataset.view(10000,1,48,48)
dataset.shape

torch.Size([35000, 1, 48, 48])

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [69]:
input_size = 2304
hidden_size = 500 
num_classes = 10
num_epochs = 3
batch_size = 2000
learning_rate = 0.001

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ## Layer 1
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 30, 5)
        #self.pool2 = nn.MaxPool2d(2, 1)
        #self.conv3 = nn.Conv2d(50, 30, 4)
        self.fc1 = nn.Linear(30 * 9 * 9, 90)
        #self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(90, 6)

    def forward(self, x):
        #Layer 1
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))
        #x = self.pool(F.relu(self.conv3(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 30 * 9 * 9)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        #x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [56]:
model = NeuralNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
batch_size = 5000

for epoch in range(num_epochs):
    for i in range(0,30000,batch_size):
        images = dataset[i:i + batch_size] 
        labels = all_labels[i:i + batch_size] 
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        #images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #if (i+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{int(i/batch_size + 1)}], Loss: {loss.item():.4f}')

Epoch [1/3], Step [1], Loss: 1.3279
Epoch [1/3], Step [2], Loss: 1.2930
Epoch [1/3], Step [3], Loss: 1.3263
Epoch [1/3], Step [4], Loss: 1.3070
Epoch [1/3], Step [5], Loss: 1.3025
Epoch [1/3], Step [6], Loss: 1.2816
Epoch [2/3], Step [1], Loss: 1.3125
Epoch [2/3], Step [2], Loss: 1.2775
Epoch [2/3], Step [3], Loss: 1.3215
Epoch [2/3], Step [4], Loss: 1.2783
Epoch [2/3], Step [5], Loss: 1.2807
Epoch [2/3], Step [6], Loss: 1.2755
Epoch [3/3], Step [1], Loss: 1.2935
Epoch [3/3], Step [2], Loss: 1.2625
Epoch [3/3], Step [3], Loss: 1.2977
Epoch [3/3], Step [4], Loss: 1.2610
Epoch [3/3], Step [5], Loss: 1.2645
Epoch [3/3], Step [6], Loss: 1.2519


In [71]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(30000, 35000, batch_size):
        #images = images.reshape(-1, 28*28).to(device)
        images = dataset[i:i + batch_size] 
        labels = all_labels[i:i + batch_size] 
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 45.12 %


Epochs done = 21